[a,b] = [-1, 1];  

q(x) = 1/eps, eps = 0.05, 

f(x) = (1/eps + pi^2)cos(pi*x), 

u = cos(pi*x) + exp((x-1)/eps^(1/2)) + exp(-(x+1)/eps^(1/2))

In [10]:
import numpy as np
np.set_printoptions(precision=10, suppress=True, linewidth=160)

In [134]:
a, b = -1, 1
eps = 0.05

q = 1/eps

f = lambda x: (1/eps + np.pi**2)*np.cos(np.pi*x) 

u = lambda x: np.cos(np.pi*x) + np.exp(((x-1)/eps**(1/2))) + np.exp(-(x+1)/eps**(1/2)) #точное решение


ua, ub = u(-1), u(1)
print(ua, ub)

0.00013048243864866027 0.00013048243864862815


In [154]:
N = 50
h = (b-a)/N
nodes = np.arange(a,b+h,h)
fi = np.vectorize(f)(nodes)
qi = q
ui = np.vectorize(u)(nodes)
#print(nodes)

AA = np.zeros((N+1, N+1))
AA[0, 0] = 1
AA[-1, -1] = 1
j = 0
for i in range(1, len(AA)-1):
    AA[i][j] = -1/(h**2)
    AA[i][j+1] = 2/(h**2)+qi
    AA[i][j+2] = -1/(h**2)
    j+=1
#print(AA)

#нижняя диагональ
A = np.zeros(N+1)
A[1:N] = -1/(h**2)
print('A:', A, len(A))

#главная диагональ
B = np.zeros(N+1)
B[0], B[-1] = 1, 1
B[1:N] = 2/(h**2)+qi
print('B:', B, len(B))

#верхняя диагональ
C = np.zeros(N+1)
C[1:N] = -1/(h**2)
print('C:', C, len(C))

#правая часть системы
F = np.zeros(N+1)
F[0], F[-1] = ua, ub
#F[1], F[-2] = fi[1] + ua/(h**2), fi[-2] + ub/(h**2)
F[1: N] = fi[1: N]
print('F:', F, len(F))


A: [   0. -625. -625. -625. -625. -625. -625. -625. -625. -625. -625. -625. -625. -625. -625. -625. -625. -625. -625. -625. -625. -625. -625. -625. -625. -625.
 -625. -625. -625. -625. -625. -625. -625. -625. -625. -625. -625. -625. -625. -625. -625. -625. -625. -625. -625. -625. -625. -625. -625. -625.    0.] 51
B: [   1. 1270. 1270. 1270. 1270. 1270. 1270. 1270. 1270. 1270. 1270. 1270. 1270. 1270. 1270. 1270. 1270. 1270. 1270. 1270. 1270. 1270. 1270. 1270. 1270. 1270.
 1270. 1270. 1270. 1270. 1270. 1270. 1270. 1270. 1270. 1270. 1270. 1270. 1270. 1270. 1270. 1270. 1270. 1270. 1270. 1270. 1270. 1270. 1270. 1270.    1.] 51
C: [   0. -625. -625. -625. -625. -625. -625. -625. -625. -625. -625. -625. -625. -625. -625. -625. -625. -625. -625. -625. -625. -625. -625. -625. -625. -625.
 -625. -625. -625. -625. -625. -625. -625. -625. -625. -625. -625. -625. -625. -625. -625. -625. -625. -625. -625. -625. -625. -625. -625. -625.    0.] 51
F: [  0.0001304824 -29.6340736488 -28.9311958525 -27.77

## Метод прогонки

In [31]:
def progonka(A, B, C, F):
    """
    метод прогонки,
    A, B, C - диагональ под главной, главная дагональ и диагональ над главной соответственно,
    F - правая часть системы
    """
    n = len(A)
    alpha = np.zeros(n)
    beta = np.zeros(n)
    x = np.zeros(n)
    alpha[0] = -C[0]/B[0]
    beta[0] = F[0]/B[0]

    for i in range(1, n):
        alpha[i] = -C[i-1]/(A[i-1]*alpha[i-1] + B[i-1])
        beta[i] = (F[i-1] - A[i-1]*beta[i-1])/(A[i-1]*alpha[i-1] + B[i-1])

    x[n-1] = (F[n-1] - A[n-1]*beta[n-1])/(A[n-1]*alpha[n-1] + B[n-1])

    for i in range(n-1, 0, -1):
        x[i-1] = alpha[i]*x[i] + beta[i]

    return np.array(x)


In [235]:
ui

array([ 0.0001304824, -0.3114451293, -0.3998561231, -0.3258697279, -0.141084819 ,  0.1080987806,  0.3792648763,  0.6344675381,  0.8416267901,  0.9762252075,
        1.022845782 ,  0.9762252075,  0.8416267901,  0.6344675381,  0.3792648763,  0.1080987806, -0.141084819 , -0.3258697279, -0.3998561231, -0.3114451293,
        0.0001304824])

In [236]:
progonka(A, B, C, F)

array([ 0.0001304824, -0.3099414812, -0.397924922 , -0.3238431714, -0.1389609256,  0.110431289 ,  0.3819097613,  0.637468032 ,  0.8449507797,  0.9797735075,
        1.0264741177,  0.9797735075,  0.8449507797,  0.637468032 ,  0.3819097613,  0.110431289 , -0.1389609256, -0.3238431714, -0.397924922 , -0.3099414812,
        0.0001304824])

In [237]:
max(abs(ui - progonka(A, B, C, F)))

0.003628335706417296

In [238]:
h**2

0.010000000000000002

## Метод Якоби

In [239]:
AA = np.zeros((N+1, N+1))
AA[0, 0] = 1
AA[-1, -1] = 1
j = 0
for i in range(1, len(AA)-1):
    AA[i][j] = -1/(h**2)
    AA[i][j+1] = 2/(h**2)+qi
    AA[i][j+2] = -1/(h**2)
    j+=1
print(AA)

[[   1.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.]
 [-100.  220. -100.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.]
 [   0. -100.  220. -100.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.]
 [   0.    0. -100.  220. -100.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.]
 [   0.    0.    0. -100.  220. -100.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.]
 [   0.    0.    0.    0. -100.  220. -100.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.]
 [   0.    0.    0.    0.    0. -100.  220. -100.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.]
 [   0.    0.    0.    0.    0.    0. -100.  220. -100.    0.    0.    0.    0.    0.    0.    0.

In [240]:
def Jacobi(A, b, eps=0.001, x=None):
    if x is None:
        x = np.zeros(A.shape[0])
                                                                                                                                                                
    D = np.diag(A)
    R = A - np.diagflat(D)

    c = 0                                                                                                                                            
    while np.linalg.norm(A@x - b) > eps:
        c += 1
        x = (b - R@x) / D

    return (x, c)

In [241]:
x, c = Jacobi(AA, F)
print(x)
c

[ 0.0001304824 -0.3099435632 -0.3979290265 -0.3238492139 -0.1389687331  0.1104218772  0.3818990147  0.637456172   0.8449381459  0.9797603602  1.0264608335
  0.9797603602  0.8449381459  0.637456172   0.3818990147  0.1104218772 -0.1389687331 -0.3238492139 -0.3979290265 -0.3099435632  0.0001304824]


99

In [242]:
ui

array([ 0.0001304824, -0.3114451293, -0.3998561231, -0.3258697279, -0.141084819 ,  0.1080987806,  0.3792648763,  0.6344675381,  0.8416267901,  0.9762252075,
        1.022845782 ,  0.9762252075,  0.8416267901,  0.6344675381,  0.3792648763,  0.1080987806, -0.141084819 , -0.3258697279, -0.3998561231, -0.3114451293,
        0.0001304824])

In [243]:
max(abs(x - ui))

0.0036150515522122006

In [244]:
np.linalg.norm(AA@x - F)

0.0009446992926631771

## Метод релаксации

In [53]:
def Relaxation(A, b, w=1, eps=0.001, x=None):
    n = len(A)
    if x is None:
        x = np.zeros(n)
        
    c = 0
    while np.linalg.norm(A@x - b) > eps:
        c += 1
        for i in range(n):
            s1 = -sum(A[i][j] * x[j] for j in range(i))/A[i][i]
            s2 = -sum(A[i][j] * x[j] for j in range(i + 1, n))/A[i][i]
            x[i] = (1-w)*x[i] +  w*(b[i]/A[i][i] + s1 + s2)

    return (x, c)

In [55]:
x, c = Relaxation(AA, F, 1.4)
print(c, max(abs(x - ui)))

2239 3.1926459390652084e-05


In [56]:
bestw, minc = 1, c
for w in np.arange(0.1, 2, 0.1):
    x, c = Relaxation(AA, F, w)
    if (c <= minc):
        bestw = w
        minc = c

    print(f"w={round(w, 1)},    loss={max(abs(x - ui))},    c={c}")

print(bestw, minc)

KeyboardInterrupt: 

## Метод наискорейшего спуска

In [57]:
def GradDesc(A, b, eps=0.01, x=None):
    n = len(A)
    if x is None:
        x = np.zeros(n)
        
    c = 0
    r = A@x - b #вектор невязки
    while np.linalg.norm(r) > eps:
        c += 1
        alpha = (r.T@r)/(A@r@r.T)
        x = x - alpha*r
        r = A@x - b

    return (x, c)

In [58]:
x, c = GradDesc(AA, F)
print(c, max(abs(x - ui)))

8646 8.435154773076642e-05


## Метод сопряженных градиентов

In [157]:
def ConjGrad(A, b, eps=0.001, x=None):
    n = len(A)
    if x is None:
        x = np.zeros(n)
        
    r0 = b - A@x
    ro0 = r0@r0.T
    p1 = r0
    q1 = A@p1

    alpha1 = ro0/(p1@q1.T)
    x = x + alpha1*p1
    r1 = r0 - alpha1*q1
    k = 1
    while np.linalg.norm(A@x - b) > eps:
        k += 1
        ro1 = r1@r1.T
        beta1 = ro1/ro0
        p2 = r1 + beta1*p1
        q2 = A@p2

        alpha1 = ro1/(p2@q2.T)
        x = x + alpha1*p2
        r1 = r1 - alpha1*q2

        r0 = r1
        ro0 = ro1
        p1 = p2
        q1 = q2

    return (x, k)

In [158]:
x, k = ConjGrad(AA, F)
print(k, max(abs(x - ui)))

25 0.000578263981932059


In [15]:
a = np.array([1, 3, -3, 2, 4, 8])
b = np.array([-2, 3, 0, 1, 3, 6])
np.corrcoef(a, b)

array([[1.          , 0.8228584085],
       [0.8228584085, 1.          ]])